In [1]:
import pathlib

In [2]:
path = pathlib.Path()
path

WindowsPath('.')

In [3]:
path.absolute()

WindowsPath('c:/workspace_deep_learning')

In [4]:
p = path / "data" / "fruit"
p.absolute()

WindowsPath('c:/workspace_deep_learning/data/fruit')

In [5]:
for i in p.iterdir():
    list(i.glob("**/*.jpg"))

1. 이미지 경로를 설정
2. 디렉토리 이름을 별도로 가져와야 함
3. 각 디렉토리에 맞춰서 이미지를 관리해야 함
4. 이미지를 텐서로 변경해야 함 (레이블을 추가해야 함)
5. 전체 이미지를 7:2:1(학습, 검증, 테스트) 나눠야 함

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms

In [7]:
def get_device():
    if torch.cuda.is_available():
        return torch.device("cuda")
    elif torch.backends.mps.is_available():
        return torch.device("mps")
    else:
        return torch.device("cpu")

In [8]:

def load_data(root_dir):
    root_path = Path(root_dir)
    # 클래스(레이블) 생성
    classes = sorted([d.name for d in root_path.iterdir() if d.is_dir()])
    class_to_idx = {class_name : idx for idx, class_name in enumerate(classes)}
    idx_to_class = {idx : class_name for idx, class_name in enumerate(classes)}

    # 이미지 파일 수집
    images = []
    labels = []
    for cls_name in classes:
        cls_dir = root_path / cls_name
        for img_path in cls_dir.glob("*.jpg"):
            images.append(img_path)
            labels.append(class_to_idx[cls_name])

    return images, labels, class_to_idx, idx_to_class

In [9]:
def get_transform(image_size=64, augment=True):
    if augment:
        transform = transforms.Compose([
            transforms.Resize((image_size, image_size)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(30),
            transforms.ColorJitter(brightness=0.2, contrast=0.2),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]
            )
        ])
    else :
        transform = transforms.Compose([
            transforms.Resize((image_size, image_size)),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]
            )
        ])
        
    return transform

In [10]:
class FruitDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform
    
    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        img_path = self.images[index]
        label = self.labels[index]
        
        image = Image.open(img_path).convert('RGB')

        if self.transform is not None:
            image = self.transform(image)
        
        label = torch.tensor(label, dtype=torch.long)
        return image, label 

In [11]:
def split_data(image, label, train_ratio = 0.8):
    n_sample = len(image)
    n_train = int(n_sample * train_ratio)

    indices = np.random.permutation(n_sample)
    train_indices = indices[:n_train]
    test_indices = indices[n_train:]

    train_images = [image[i] for i in train_indices]
    val_images = [image[i] for i in test_indices]
    train_labels = [label[i] for i in train_indices]
    val_labels = [label[i] for i in test_indices] 

    return train_images, val_images, train_labels, val_labels

In [12]:
data_dir = "data/fruit"
max_epochs = 3
batch_size = 32
image_size = 64

torch.manual_seed(42)
np.random.seed(42)

device = get_device()
print(f'{device}를 사용합니다.')

cpu를 사용합니다.


In [13]:
images, labels, class_to_idx, idx_to_class = load_data(data_dir)
train_images, val_images, train_labels, val_labels = split_data(images, labels)
transform_train = get_transform(image_size, augment=True)
transform_val = get_transform(image_size, augment=False)

train_dataset = FruitDataset(train_images, train_labels, transform_train)
val_dataset = FruitDataset(val_images, val_labels, transform_val)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


# 14장에 나오는 CNN을 해야 합니다


In [ ]:
import torch.nn as nn

class SimpleLinear(nn.Module):
    def __init__(self, num_classes=33, image_size=64):
        super(SimpleLinear, self).__init__()
        input_size = 3 * image_size * image_size
        self.fc1 = nn.Linear(input_size, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, num_classes)
        self.dropout = nn.Dropout(0.5)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.dropout(self.relu(self.fc2(x)))
        x = self.fc3(x)
        return x

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=5, device="cpu"):
    train_loss_hist = []
    train_acc_hist = []
    val_loss_hist = []
    val_acc_hist = []

    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0.0
        correct = 0
        total = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            prediction = model(images)
            loss = criterion(prediction, labels)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            _, predicted = torch.max(prediction.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        avg_loss = epoch_loss /len(train_loader)
        acc = 100 * correct / total
        train_loss_hist.append(avg_loss)
        train_acc_hist.append(acc)

        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        with torch.no_grad(): # 기울기 계산을 끔
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_avg_loss = val_loss /len(val_loader)
        val_acc = 100 * val_correct / val_total
        val_loss_hist.append(val_avg_loss)
        val_acc_hist.append(val_acc)

    return train_loss_hist, train_acc_hist, val_loss_hist, val_acc_hist